In [3]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image

In [35]:
# Use my file:
path_to_file = "himym.txt"
path_to_file_1 = 'theoffice.txt'

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = "ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 15391 characters


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

69 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 120
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "OLDER TED: Kids, I'm gonna tell you an incredible story\nThe story of how I met your mother\n- Are we being punished for s"
Target data: "LDER TED: Kids, I'm gonna tell you an incredible story\nThe story of how I met your mother\n- Are we being punished for so"


In [11]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('O')
  expected output: 27 ('L')
Step    1
  input: 27 ('L')
  expected output: 19 ('D')
Step    2
  input: 19 ('D')
  expected output: 20 ('E')
Step    3
  input: 20 ('E')
  expected output: 32 ('R')
Step    4
  input: 32 ('R')
  expected output: 1 (' ')


In [12]:
# Batch size (number of 100 word sequences to train on)
BATCH_SIZE = 64

# How many individual training steps per samples (steps)
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 120), (64, 120)), types: (tf.int64, tf.int64)>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [14]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
te
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [16]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)

In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 120, 69)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.234866


In [20]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './CheckpointData'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
history = model.fit(dataset.repeat(), epochs=40, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/40
2/2 [==============================] - 1s 687ms/step - loss: 4.2028
Epoch 2/40
2/2 [==============================] - 1s 265ms/step - loss: 6.3248
Epoch 3/40
2/2 [==============================] - 1s 293ms/step - loss: 3.6608
Epoch 4/40
2/2 [==============================] - 1s 291ms/step - loss: 3.8780
Epoch 5/40
2/2 [==============================] - 1s 286ms/step - loss: 3.8356
Epoch 6/40
2/2 [==============================] - 1s 306ms/step - loss: 3.7393
Epoch 7/40
2/2 [==============================] - 1s 304ms/step - loss: 3.6680
Epoch 8/40
2/2 [==============================] - 1s 330ms/step - loss: 3.5232
Epoch 9/40
2/2 [==============================] - 1s 329ms/step - loss: 3.3992
Epoch 10/40
2/2 [==============================] - 1s 327ms/step - loss: 3.2186
Epoch 11/40
2/2 [==============================] - 1s 283ms/step - loss: 3.2430
Epoch 12/40
2/2 [==============================] - 1s 319ms/step - loss: 3.1973
Epoch 13/40
2/2 [==============================] 

In [23]:
tf.train.latest_checkpoint(checkpoint_dir)

'./CheckpointData/ckpt_40'

In [24]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [33]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
        
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [34]:
print(generate_text(model, start_string=u"Begin:\n"))


Begin:
-'l wak
I Cot wey inllig fre sowe
- Nhe junit
Be dod thing the alve cous cimend
Thale we
Yon, I wouny and sond our thas Ther corve dhar phet
Gor the me
I sout be juthe os orels monw, domingtit tos ing inlir kollik
Lol don ton, hanl the as fatlon, I duren tim
He syoo lelnon be nowe, don an dores get mast or ay doret thina thit th
To, hare llat
Ang in hagse thave lo whas thing so tho bers arsy urece wow an cow mart you gire go sotha lot nom gowes
Let, thathe al tit hory, nave sarset the lod
I Youn tol hasthe the won and on'g wer your you le got son ang tos juver she to'w mhan the gorme weave I baven
Tive sey ans
- Het mave rerla some hot on that the he mast
I me fover ghand than'nl oo, I gome mit me yout gof bes wane I sonte
Tid sha hoker in Ser hrere
I Tore pibes tout
You gtet you t oor fowe erfy ond bo gore she lfore son one mo
Sem ancat mis ora lle
What they he cou gyou gome the gor you, the you gous te basthe
- Yese meidle't mike somey an buant wount etaghe gherle ho't he pils